In [135]:
import time
import os
import getpass
# from config import OPENAI_API_KEY


from langchain_community.vectorstores.azuresearch import AzureSearch
from langchain.memory import ConversationBufferWindowMemory
from langchain import LLMMathChain
from langchain.chat_models import ChatOpenAI
from langchain.agents import LLMSingleActionAgent, AgentExecutor, Tool
from langchain.agents import create_react_agent
from langchain import LLMChain
from langchain.agents.agent_toolkits import create_retriever_tool
# from output_parser import CustomOutputParser
# from prompt_template import CustomPromptTemplate
# from prompts import TEMPLATE_INSTRUCTIONS, SUFFIX
# from tools import retriever_tool


from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
load_dotenv()

True

In [136]:
vector_store_address: str = os.environ["YOUR_AZURE_SEARCH_ENDPOINT"]
vector_store_password: str = os.environ["YOUR_AZURE_SEARCH_ADMIN_KEY"]

In [137]:
if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass("Provide your Google API key here")

# print(os.environ["GOOGLE_API_KEY"])

In [138]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [139]:
index_name: str = "langchain-vector-demo"
vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [149]:
# Set OpenAI LLM and embeddings
llm_chat = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

# Set conversation memory buffer
memory = ConversationBufferWindowMemory(
    memory_key="history", k=5, return_messages=True)

memory.chat_memory.add_user_message("Hi there!")
memory.chat_memory.add_ai_message("hello :)")

print(memory.memory_key)

history


In [141]:
retriever = vector_store.as_retriever(search_kwargs={"k": 3})

retriever_tool = create_retriever_tool(
    retriever,
    name="qa_conversational_business",
    description="Searches and returns documents regarding conversational business. Input: send the user input."
)

In [142]:
tools = [
    retriever_tool
]

In [143]:
from typing import List
from langchain.agents import Tool
from langchain.schema import SystemMessage
from langchain.prompts import BaseChatPromptTemplate


# Set up a prompt template
class CustomPromptTemplate(BaseChatPromptTemplate):
    # The template to use
    prefix: str
    instructions: str
    sufix: str
    # The list of tools available
    tools: List[Tool]

    def _set_tool_description(self, tool_description, tool_name, tool_input):
        full_description = f"""{tool_description}, send this:
```json
{{"action": "{tool_name}",
"action_input": "{tool_input}"}}
```
"""
        return full_description

    def format_messages(self, **kwargs) -> str:
        # Get the intermediate steps (AgentAction, Observation tuples)
        # Format them in a particular way
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""

        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "

        # Set the agent_scratchpad variable to that value
        kwargs["agent_scratchpad"] = thoughts

        # Create a tools variable from the list of tools provided
        separator = '. Input:'
        kwargs["tools"] = "\n".join(
            [f"{self._set_tool_description(tool.description.split(separator)[0], tool.name, tool.description.split(separator)[1])}" for tool in self.tools])

        # Create a list of tool names for the tools provided
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])

        # Format the instructions replacing the variables with the values
        formatted = self.instructions.format(**kwargs)

        # Add the sufix
        formatted += self.sufix
        return [SystemMessage(content=formatted)]

In [144]:
TEMPLATE_INSTRUCTIONS = """You are an Assistant designed to assist with a wide range of tasks.

Choose one of the following tools to use based on the user input:

- {tools}

- When you need to respond to other user's utterances or generate the response from the other tools, send this:
    ```json
    {{"action": "Final Answer",
      "action_input": "the final answer to the original input question"}}
    ```
- When you need to generate the response from the other tools, send this:
    ```json
    {{"action": "Final Answer",
      "action_input": "the response from the tool in a prhase"}}
    ```
[{tool_names}]
Current conversation:
{history}
User: {input}
Assistant:

{agent_scratchpad}"""

SUFFIX = """\nRespond ONLY in JSON format!"""

In [145]:
# Set up prompt template
prompt = CustomPromptTemplate(
    prefix='',
    instructions=TEMPLATE_INSTRUCTIONS,
    sufix=SUFFIX,
    tools=tools,
    input_variables=["input", "intermediate_steps", "history"]
)

In [146]:
from langchain.agents import AgentOutputParser
from langchain.schema import AgentAction, AgentFinish
from langchain.output_parsers.json import parse_json_markdown


class CustomOutputParser(AgentOutputParser):

    def parse(self, text: str) -> any:
        try:
            # this will work IF the text is a valid JSON with action and action_input
            response = parse_json_markdown(text)
            action, action_input = response["action"], response["action_input"]
            if action == "Final Answer":
                # this means the agent is finished so we call AgentFinish
                return AgentFinish({"output": action_input}, text)
            else:
                # otherwise the agent wants to use an action, so we call AgentAction
                return AgentAction(action, action_input, text)
        except Exception:
            # sometimes the agent will return a string that is not a valid JSON
            # often this happens when the agent is finished
            # so we just return the text as the output
            return AgentFinish({"output": text}, text)

In [147]:

# Set up output parser
output_parser = CustomOutputParser()

In [148]:

# Set up the agent
# llm_chain = LLMChain(llm=llm_chat, prompt=prompt)
# tool_names = [tool.name for tool in tools]

# agent = LLMSingleActionAgent(
#     llm_chain=llm_chain,
#     output_parser=output_parser,
#     stop=["\nObservation:"],
#     allowed_tools=tool_names,
# )
# agent_executor = AgentExecutor.from_agent_and_tools(
#     agent=agent, tools=tools, max_iterations=3)

agent = create_react_agent(llm_chat, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# agent_executor = initialize_agent(tools=tools, llm=llm_chat,
#                                   agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True, memory=memory)




ValueError: Prompt missing required variables: {'agent_scratchpad', 'tool_names', 'tools'}

In [ ]:

# response = agent_executor.invoke({"input": "does the Northwind Health Plus covers hearning exams?", "history":[]})
# print('Assistant: ' + response)

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
response = agent_executor.invoke(
    {
        "history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
        "input": "does the Northwind Health Plus covers hearning exams?",
    }
)
print('Assistant: ' + response)



> Entering new AgentExecutor chain...


IndexError: pop from empty list